# Interior point methods

In [1]:
using LinearAlgebra
using SparseArrays

# local modules
push!(LOAD_PATH, realpath("../code"))
using lp_problem
using lp_standard_form_converter
using lp_read_mps

In [2]:
# function read_file_to_string(filepath)
#     mps_string = open(filepath, "r") do f
#         read(f, String)
#     end
#     return mps_string
# end

In [ ]:
filepath ="../../check/benchmarks/mps_files/test.mps"
println(read_file_to_string(filepath))

In [ ]:
lp = read_mps_from_file_mip("../../check/benchmarks/mps_files/test.mps")

In [ ]:
print(read_mps_from_file("../../check/benchmarks/mps_files/test.mps"))

In [6]:
struct Parameters
    maxIter::Int
    tol::Float64
    eta::Float64
    verbose::Int
end

struct IPMSolver
    mip_prob::MIPProblem
    pars::Parameters
    
    function IPMSolver(mip_prob::MIPProblem)
        return new(mip_prob, Parameters(100, 1e-6, 0.95, 1))
    end
    
    function IPMSolver(mip_prob::MIPProblem, pars::Parameters)
        return new(mip_prob, pars)
    end
    
    function IPMSolver(A::SparseMatrixCSC{Float64, Int64}, b::Vector{Float64}, c::Vector{Float64}, l::Vector{Float64}, u::Vector{Float64}, vars::Vector{String}, variable_types::Vector{Symbol}, constraint_types::Vector{Char})
        mip_prob = MIPProblem(true, c, A, b, l, u, vars, variable_types, constraint_types)
        return new(mip_prob, Parameters(100, 1e-6, 0.95, 1))
    end
end


In [ ]:


# Interior Point Method for solving LPProblem
function interior_point_solver(lp::MIPProblem; tol=1e-6, max_iter=100)
    # Unpack problem data
    c = lp.is_minimize ? lp.c : -lp.c  # Adjust for maximization problems
    A = lp.A
    b = lp.b
    m, n = size(A)

    # Initialize variables
    x = ones(n)  # Primal variables (start with feasible guess)
    s = ones(n)  # Slack variables
    y = zeros(m)  # Dual variables
    μ = 1.0  # Barrier parameter

    for iter in 1:max_iter
        # Compute residuals
        r_dual = A' * y + s - c
        r_cent = s .* x - μ * ones(n)
        r_pri = A * x - b

        # Check convergence
        if norm(r_dual, Inf) < tol && norm(r_pri, Inf) < tol && norm(r_cent, Inf) < tol
            println("Converged in $iter iterations.")
            return x
        end

        # Form the KKT matrix
        D_inv = Diagonal(1.0 ./ x)
        M = A * D_inv * A'
        rhs = -(r_pri + A * D_inv * (r_cent - D_inv * r_dual))

        # Solve for the dual variables
        Δy = M \ rhs

        # Solve for the primal and slack variables
        Δs = r_dual - A' * Δy
        Δx = -x .+ D_inv * (r_cent - Δs .* x)

        # Line search for feasibility
        α = 1.0
        for i in 1:n
            if Δx[i] < 0
                α = min(α, -x[i] / Δx[i])
            end
            if Δs[i] < 0
                α = min(α, -s[i] / Δs[i])
            end
        end
        α = 0.99 * α  # Take a step just inside the feasible region

        # Update variables
        x += α * Δx
        s += α * Δs
        y += α * Δy

        # Update the barrier parameter
        μ *= 0.9  # Reduce barrier parameter
    end

    println("Maximum iterations reached without convergence.")
    return x
end


In [ ]:
# # Example usage
# A = sparse([1.0 2.0; 3.0 4.0])
# b = [1.0, 1.0]
# c = [-1.0, -2.0]
# l = [0.0, 0.0]
# u = [Inf, Inf]
# vars = ["x1", "x2"]
# variable_types = [:Continuous, :Continuous]
# constraint_types = ['L', 'L']

# lp = LPProblem(true, c, A, b, l, u, vars, variable_types, constraint_types)

# Solve the LP problem
solution = interior_point_solver(lp)
println("Optimal solution: ", solution)


In [ ]:
# Solve the LP problem
solution = interior_point_solver(lp, tol=1e-8, max_iter=5000)
println("Optimal solution: ", solution)

## 

# References

Karmarkar, N. A new polynomial-time algorithm for linear programming. Combinatorica 4, 373–395 (1984). https://doi.org/10.1007/BF02579150


```bibtex
@book{SIAMB0000215,
 title={Primal-dual interior-point methods},
 author={Wright, Stephen J.},
 year={1997},
 publisher={Society for Industrial and Applied Mathematics},
 pages={310},
 isbn={9780898713824},
 url={https://portal-igpublish-com.ezproxy.newcastle.edu.au/iglibrary/search/SIAMB0000215.html}
}
```

Megiddo, N. On the complexity of computing an approximate solution for combinatorial optimization problems. SIAM Journal on Computing 18, 1189-1198 (1989). https://doi.org/10.1137/0218074

Forsgren, A., & Gill, P. E. (2007). A Matlab Implementation of a Primal-Dual Interior Point Method for Linear Programming. Computational Optimization and Applications, 36(2-3), 209-241. https://doi.org/10.1007/s10589-006-9005-9